# Data Wrangling

Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

## Merging Data

Suppose we had data from two different sources and wanted to combine them to look at relationships between variables and overall create a more comprehensive dataset. To do this, we'd have to **merge** the two datasets together. That is, we would need to make sure that the observations match on certain characteristics to make sure that it's the same entity (such as individual, county, state, etc.). Then, we'd need to get the variables from both datasets and make sure they are included in the full combined table for each observation.

With all the different methods of data collection available nowadays, this is becoming more and more common. In this section, we'll go over how to handle data from multiple sources, and why we might want to use them together. 

In [22]:
from requests import get

import numpy as np 
import pandas as pd 

## Census Data
Let's first use the Census API to get some data to analyze. We'll bring in our census key from a text file first. Remember to copy and paste that file into this folder so that the following piece of code works.

In [23]:
with open('census-key.txt', 'r') as f:
    census_key = f.readline()

Let's start by bringing in a set of variables from the 2022 American Community Survey (ACS) Data Profiles tables. We'll look at some employment and education characteristics of people by state. The variables we'll want to pull are:
- `NAME`: State name
- `DP02_0001E`: Total number of households in the state
- `DP03_0087E`: Mean income of people in the state
- `DP03_0002PE`: Percent of people 16 years and older in labor force
- `DP02_0068PE`: Percent of people 25 years and older with a Bachelor's Degree or higher
- `DP02_0066PE`: Percent of people 25 years and older with a Graduate or Professional degree

Note that we are going to be using the Data Profile table (note the DP at the beginning of each variable name) so the base url needs to be for that particular type of table. 

In [24]:
year = 2022
census_base_url = f'https://api.census.gov/data/{year}/acs/acs1/profile'

census_params = {'get':'NAME,DP02_0001E,DP03_0087E,DP03_0002PE,DP02_0068PE,DP02_0066PE',
                 'for':'state:*',
                 'key':census_key}

r = get(census_base_url, params = census_params)
r.status_code

200

In [25]:
# Removing Puerto Rico due to lack of data.
people_by_state = r.json()[:-1]

In [26]:
keys = ['state', 'num_households','mean_income','percent_employed','percent_bachelors','percent_graduate']

In [27]:
census_dict = {key:[float(state[keys.index(key)]) for state in people_by_state[1:]] for key in keys[1:]}
census_dict['state'] = [state[0] for state in people_by_state[1:]]
census_df = pd.DataFrame(census_dict)
census_df.head()


,num_households,mean_income,percent_employed,percent_bachelors,percent_graduate,state
0,2016448.0,100785.0,58.6,28.8,11.3,Alabama
1,274574.0,124663.0,67.0,30.6,11.3,Alaska
2,2850377.0,116717.0,60.7,33.0,12.5,Arizona
3,1216207.0,92935.0,57.8,25.4,9.7,Arkansas
4,13550586.0,147628.0,63.9,37.0,14.4,California


<font color ='red'>**Question 1: Read in the data from `states.csv` as a DataFrame called `states`. Look at the first few rows. What does it look like this dataset contains?**</font>

<font color ='red'>**Question 2: How many different regions are there within `states`? How many states are in each region?**</font>

## Merging Data

Suppose you wanted to know what the distribution of mean incomes are like in the different regions of the US. The Census data has the information on mean incomes, but we also need to add information about the region, which is in the `states` DataFrame. To use the information in these data together, we need to **merge** them. 

To merge the datasets together, we need to make sure we have the appropriate variable(s) to merge on. In this case, we want to add the state abbreviation and region to the information we got from the Census API. Since each row represents a state, we want to match on the **unique state identifier that is common in both datasets.** This would be the state name here, since that is the variable that is the same in both. Note that this can vary depending on the datasets you want to join together. Make sure you have a good idea of what each row represents in your dataset when you are doing these types of merges.

Using the `merge` method, we provide a datasets to merge with as well as the variables we want to merge on.

In [28]:
merged_data = census_df.merge(states, left_on = 'state', right_on = 'State')
merged_data.head()

NameError: name 'states' is not defined

<font color ='red'>**Question 3: How many rows are there in `merge_data`? Does this differ from `census_df`?**</font>

You'll notice that the merged data no longer contains information about Washington D.C. This is because the `states` data doesn't have D.C. in it. The merge omitted any rows that didn't have a matching value in both datasets. This is called an `inner join`. If you had wanted to keep DC in and just leave the values from the `state` DataFrame as missing for D.C., we could have done a `left join` 

In [ ]:
left_join_data = census_df.merge(states, left_on = 'state', right_on = 'State', how = 'left')
left_join_data.shape

In [ ]:
left_join_data[left_join_data.state == 'District of Columbia']

### Group Operations

Now that we have merged our datasets, we can start looking at the breakdown of the Census variables by region. We could do this by looking at four different subsets of the `Region` variable, but repeating the same analysis four times can be tedious. Instead, we can use the `groupby` method and use what is called the split-apply-combine workflow.

First, we use `groupby` to create the groups that we to use. In this case, we want to get some statistics by region, so we use `groupby('Region')`. Then, we can use these groups to apply whatever action we want to, and Python will combine the results and provide the full summary split by each group. Let's take a look at the code. 

In [ ]:
merged_data[['Region','mean_income']].groupby('Region').describe()

We are able to get the various summary statistics for `mean_income` using `describe` split up by each `Region`. This allows us to make comparisons between `Region` without needing to calculate all of these manually. 

We can also do the same for plots.

In [ ]:
merged_data[['Region','mean_income']].groupby("Region").boxplot()

(Although in this case using "by" might be more appropriate)

In [ ]:
merged_data[['Region','mean_income']].boxplot(by="Region")

This helps us see the distribution of mean income by Region. 

<font color ='red'>**Question 4: What are the differences in percent of people with a graduate or professional degree by region of the US?**</font>

### Joining Another Dataset

Consider the `anxiety.csv` file. This data comes from a daily survey administered through Facebook asking a variety of questions during the COVID-19 pandemic. The value reported in `anxiety.csv` is the "estimated percentage of respondents who reported feeling 'nervous, anxious, or on edge' for most or all of the past 7 days." The percentage was calculated at the state level, and the dataset contains responses over a number of days between May 1, 2022 and June 25, 2022. 

> This data was obtained using the Delphi Epidata API (https://cmu-delphi.github.io/delphi-epidata/) and has been cleaned to fit this exercise. You can also try accessing the API yourself to get the data!

Let's say we want to look at whether factors such as employment rate and level of education is associated with higher levels of reported anxiety during this time period. The data for employment and education are in the Census data, while the data about anxiety are in the `anxiety` data file. We'll need to join them together to use them together. 

In [ ]:
anxiety = pd.read_csv('anxiety.csv')
anxiety.head()

Note that the `geo_value` variable has the 2-letter state code, but it is all lowercase instead of being uppercase like it is in our Census data. We will first need to make sure that they match exactly so that the merge can work properly.

In [ ]:
merged_data.Abbreviation = merged_data.Abbreviation.str.lower()
merged_data.head()

<font color ='red'>**Question 5: Join `merged_data` with `anxiety`. Call the new dataset `full_data`.**</font>

<font color ='red'>**Question 6: Consider only data from May 1, 2022. What is the relationship between anxiety and percentage of people employed within each state? Use a scatterplot to look at the relationship.**</font>

<font color ='red'>**Question 7: Again using only data from May 1, 2022, are there differences in distribution of proportion of people who reported anxiety by region of the US?**</font>